In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../')
import logging
import pandas as pd
logging.basicConfig(
    level=logging.DEBUG,  # Set the logging level
    format='%(asctime)s - %(levelname)s - %(message)s',  # Specify the log message format
    datefmt='%Y-%m-%d %H:%M:%S',  # Specify the date format
    handlers=[
        #logging.FileHandler('app.log'),  # Log messages to a file
        logging.StreamHandler()  # Also output log messages to the console
    ]
)
logger = logging.getLogger(__name__)

In [29]:
node_features = pd.read_csv('/mnt/data/elliptic_bitcoin_dataset/elliptic_txs_features.csv', header=None)
edges = pd.read_csv('/mnt/data/elliptic_bitcoin_dataset/elliptic_txs_edgelist.csv')
classes = pd.read_csv('/mnt/data/elliptic_bitcoin_dataset/elliptic_txs_classes.csv')
classes.index = classes.index.astype(str)

In [30]:
nodes = node_features.merge(classes, left_on=0, right_on='txId', how='left')

In [31]:
# renumber txID to be 0-indexed and continuous
nodes['txId'] = [str(i) for i in range(nodes.shape[0])]
nodes.head()

,0,1,2,3,4,5,6,7,8,9,...,159,160,161,162,163,164,165,166,txId,class
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,0,unknown
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,1,unknown
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792,2,unknown
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792,3,2
4,230460314,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117,4,unknown


In [38]:
nodes.to_csv('/mnt/data/elliptic_bitcoin_dataset/elliptic_txs_nodes.csv', index=False)

In [33]:
edges.head()

,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206


In [36]:
# use column 0 and txID in nodes to update the IDs in edges df
n_map = nodes[[0, 'txId']]
print(n_map.head())
edges2 = edges.merge(n_map, left_on='txId1', right_on=0, how='left')
edges2 = edges2.drop(columns=['txId1'])
edges2 = edges2.rename(columns={'txId': 'txId1'})
print(edges2.head())
edges2 = edges2.merge(n_map, left_on='txId2', right_on=0, how='left')
edges2 = edges2.drop(columns=['txId2'])
edges2 = edges2.rename(columns={'txId': 'txId2'})
# drop 0_x and 0_y columns
edges2 = edges2.drop(columns=['0'+'_x', '0'+'_y'])
edges2.head()

           0 txId
0  230425980    0
1    5530458    1
2  232022460    2
3  232438397    3
4  230460314    4
       txId2          0 txId1
0    5530458  230425980     0
1  232438397  232022460     2
2  230459870  230460314     4
3  230595899  230333930     6
4  232029206  232013274     8


,txId1,txId2
0,0,1
1,2,3
2,4,5
3,6,7
4,8,9


In [37]:
edges2.to_csv('/mnt/data/elliptic_bitcoin_dataset/elliptic_txs_edgelist_mapped.csv', index=False)

In [6]:
nodes.drop(columns=[0], inplace=True)
nodes.head()

,1,2,3,4,5,6,7,8,9,10,...,159,160,161,162,163,164,165,166,txId,class
0,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,-0.167933,...,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,0,unknown
1,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,-0.167948,...,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,1,unknown
2,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,-0.168576,...,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792,2,unknown
3,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,-0.115831,...,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792,3,2
4,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,0.041399,...,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117,4,unknown


In [35]:
nodes.to_csv('/mnt/data/elliptic_bitcoin_dataset/elliptic_txs_nodes.csv', index=False)

In [36]:
nodes = pd.read_csv('/mnt/data/elliptic_bitcoin_dataset/elliptic_txs_nodes.csv')
print(nodes.index)
print(nodes.columns)
nodes.head()

RangeIndex(start=0, stop=203769, step=1)
Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10',
       ...
       '159', '160', '161', '162', '163', '164', '165', '166', 'txId',
       'class'],
      dtype='object', length=168)


,1,2,3,4,5,6,7,8,9,10,...,159,160,161,162,163,164,165,166,txId,class
0,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,-0.167933,...,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,230425980,unknown
1,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,-0.167948,...,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792,5530458,unknown
2,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,-0.168576,...,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792,232022460,unknown
3,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,-0.115831,...,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792,232438397,2
4,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,0.041399,...,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117,230460314,unknown


In [28]:
# statistics about the class column
classes['class'].value_counts()

class
unknown    157205
2           42019
1            4545
Name: count, dtype: int64

In [5]:
node_features.head()

,0,1,2,3,4,5,6,7,8,9,...,157,158,159,160,161,162,163,164,165,166
0,230425980,1,-0.171469,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162097,...,-0.562153,-0.600999,1.461330,1.461369,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
1,5530458,1,-0.171484,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162112,...,0.947382,0.673103,-0.979074,-0.978556,0.018279,-0.087490,-0.131155,-0.097524,-0.120613,-0.119792
2,232022460,1,-0.172107,-0.184668,-1.201369,-0.121970,-0.043875,-0.113002,-0.061584,-0.162749,...,0.670883,0.439728,-0.979074,-0.978556,-0.098889,-0.106715,-0.131155,-0.183671,-0.120613,-0.119792
3,232438397,1,0.163054,1.963790,-0.646376,12.409294,-0.063725,9.782742,12.414558,-0.163645,...,-0.577099,-0.613614,0.241128,0.241406,1.072793,0.085530,-0.131155,0.677799,-0.120613,-0.119792
4,230460314,1,1.011523,-0.081127,-1.201369,1.153668,0.333276,1.312656,-0.061584,-0.163523,...,-0.511871,-0.400422,0.517257,0.579382,0.018279,0.277775,0.326394,1.293750,0.178136,0.179117


In [26]:
edges.head()

,txId1,txId2
0,230425980,5530458
1,232022460,232438397
2,230460314,230459870
3,230333930,230595899
4,232013274,232029206


In [27]:
# get unique node ids from txId1 and txId2
tx_ids = pd.concat([nodes['txId1'], nodes['txId2']]).unique()
print(len(tx_ids))
print(len(node_features))

203769
203769


In [2]:
nodes = pd.read_csv('/mnt/data/elliptic_bitcoin_dataset/elliptic_txs_nodes.csv')
edges = pd.read_csv('/mnt/data/elliptic_bitcoin_dataset/elliptic_txs_edgelist_mapped.csv')
print(nodes.head())
print(edges.head())

           0  1         2         3         4          5         6         7  \
0  230425980  1 -0.171469 -0.184668 -1.201369  -0.121970 -0.043875 -0.113002   
1    5530458  1 -0.171484 -0.184668 -1.201369  -0.121970 -0.043875 -0.113002   
2  232022460  1 -0.172107 -0.184668 -1.201369  -0.121970 -0.043875 -0.113002   
3  232438397  1  0.163054  1.963790 -0.646376  12.409294 -0.063725  9.782742   
4  230460314  1  1.011523 -0.081127 -1.201369   1.153668  0.333276  1.312656   

           8         9  ...       159       160       161       162       163  \
0  -0.061584 -0.162097  ...  1.461330  1.461369  0.018279 -0.087490 -0.131155   
1  -0.061584 -0.162112  ... -0.979074 -0.978556  0.018279 -0.087490 -0.131155   
2  -0.061584 -0.162749  ... -0.979074 -0.978556 -0.098889 -0.106715 -0.131155   
3  12.414558 -0.163645  ...  0.241128  0.241406  1.072793  0.085530 -0.131155   
4  -0.061584 -0.163523  ...  0.517257  0.579382  0.018279  0.277775  0.326394   

        164       165       166 

In [4]:
dummy_edges = edges.head(1000)
unique_ids = pd.concat([dummy_edges['txId1'], dummy_edges['txId2']]).unique()
id_map = {old_id: new_id for new_id, old_id in enumerate(unique_ids)}
dummy_edges['txId1'] = dummy_edges['txId1'].apply(lambda x: id_map[x])
dummy_edges['txId2'] = dummy_edges['txId2'].apply(lambda x: id_map[x])

mask = nodes["txId"].isin(unique_ids)
dummy_nodes = nodes[mask]
dummy_nodes['txId'] = dummy_nodes['txId'].apply(lambda x: id_map[x])

/tmp/ipykernel_933638/560359218.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dummy_edges['txId1'] = dummy_edges['txId1'].apply(lambda x: id_map[x])
/tmp/ipykernel_933638/560359218.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dummy_edges['txId2'] = dummy_edges['txId2'].apply(lambda x: id_map[x])
/tmp/ipykernel_933638/560359218.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats

In [7]:
print(dummy_nodes.head())
print(dummy_edges.head())
print(len(dummy_nodes))
print(len(dummy_edges))
dummy_nodes.to_csv('/mnt/data/elliptic_bitcoin_dataset-dummy/elliptic_txs_nodes.csv', index=False)
dummy_edges.to_csv('/mnt/data/elliptic_bitcoin_dataset-dummy/elliptic_txs_edgelist_mapped.csv', index=False)

           0  1         2         3         4          5         6         7  \
0  230425980  1 -0.171469 -0.184668 -1.201369  -0.121970 -0.043875 -0.113002   
1    5530458  1 -0.171484 -0.184668 -1.201369  -0.121970 -0.043875 -0.113002   
2  232022460  1 -0.172107 -0.184668 -1.201369  -0.121970 -0.043875 -0.113002   
3  232438397  1  0.163054  1.963790 -0.646376  12.409294 -0.063725  9.782742   
4  230460314  1  1.011523 -0.081127 -1.201369   1.153668  0.333276  1.312656   

           8         9  ...       159       160       161       162       163  \
0  -0.061584 -0.162097  ...  1.461330  1.461369  0.018279 -0.087490 -0.131155   
1  -0.061584 -0.162112  ... -0.979074 -0.978556  0.018279 -0.087490 -0.131155   
2  -0.061584 -0.162749  ... -0.979074 -0.978556 -0.098889 -0.106715 -0.131155   
3  12.414558 -0.163645  ...  0.241128  0.241406  1.072793  0.085530 -0.131155   
4  -0.061584 -0.163523  ...  0.517257  0.579382  0.018279  0.277775  0.326394   

        164       165       166 

In [5]:
from src.datasets import EllipticBitcoin

dataset = EllipticBitcoin(root='/mnt/data/elliptic_bitcoin_dataset/')

2024-08-10 10:30:53 - INFO - Creating graph...


                   1             2             3             4             5  \
count  203769.000000  2.037690e+05  2.037690e+05  2.037690e+05  2.037690e+05   
mean       23.843961  2.454849e-17 -7.531922e-18  2.231681e-17  6.241732e-17   
std        15.172170  1.000002e+00  1.000002e+00  1.000002e+00  1.000002e+00   
min         1.000000 -1.729826e-01 -2.105526e-01 -1.756361e+00 -1.219696e-01   
25%         9.000000 -1.725317e-01 -1.803266e-01 -1.201369e+00 -1.219696e-01   
50%        23.000000 -1.692045e-01 -1.328975e-01  4.636092e-01 -1.219696e-01   
75%        38.000000 -1.318553e-01 -5.524241e-02  1.018602e+00 -1.219696e-01   
max        49.000000  7.168197e+01  7.359505e+01  2.683580e+00  4.902760e+01   

                  6             7             8             9            10  \
count  2.037690e+05  2.037690e+05  2.037690e+05  2.037690e+05  2.037690e+05   
mean   7.653968e-17  5.858162e-17  3.068561e-17  3.570689e-17  4.630738e-17   
std    1.000002e+00  1.000002e+00  1.00000

: 